In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
def get_data(week_nums):
    filename = 'turnstile_{}.csv'
    dfs = []
    for week_num in week_nums:
        file_name = filename.format(week_num)
        dfs.append(pd.read_csv(file_name))
    return pd.concat(dfs)

In [3]:
week_nums = [170429, 170506, 170513, 170520, 170527, 170603]
subwaydf = get_data(week_nums)

In [4]:
#Old way we called in a single file
#filename = 'subway_data.txt'
#subwaydf = pd.read_csv(filename)

In [5]:
subwaydf.columns = subwaydf.columns.str.strip() #strip whitespace from end of column names

In [6]:
subwaydf['datetime'] = subwaydf['DATE'] + ' ' + subwaydf['TIME']

In [7]:
subwaydf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1179307 entries, 0 to 195957
Data columns (total 12 columns):
C/A         1179307 non-null object
UNIT        1179307 non-null object
SCP         1179307 non-null object
STATION     1179307 non-null object
LINENAME    1179307 non-null object
DIVISION    1179307 non-null object
DATE        1179307 non-null object
TIME        1179307 non-null object
DESC        1179307 non-null object
ENTRIES     1179307 non-null int64
EXITS       1179307 non-null int64
datetime    1179307 non-null object
dtypes: int64(2), object(10)
memory usage: 117.0+ MB


In [8]:
#Join date and time columns and combine to be new datetime, then make the new column a datetime object
subwaydf['datetime'] = pd.to_datetime(subwaydf['datetime'])

In [9]:
#Sort the subway data by station, unit, terminal and by date
subwaydf = subwaydf.sort_values(['STATION', 'UNIT', 'SCP', 'datetime'])
subwaydf.reset_index(inplace = True)

In [10]:
#create a pre and next iterator column so we can do some comparisons across cells
subwaydf['entries_pre'] = subwaydf['ENTRIES'].shift(1)
subwaydf['entries_next'] = subwaydf['ENTRIES'].shift(-1)

In [11]:
#If the content of an individual cell of entries is out of order, ie. index resets or jumps high
#Return the mean of the cells before and after. Useful to catch and clean abnormal data one offs

def convertToMean(row):
    a = row['entries_pre']
    b = row.ENTRIES
    c = row.entries_next
    if((a<=b<=c) or (a>=b>=c)):
        return b
    elif((b>a) & (b>c) & (a<c)):
        return ((a+c)//2)
    elif((b<a) & (b<c) & (a<c)):
        return ((a+c)//2)
    else:
        return b

In [12]:
#Apply the previous function create a column of fixed entries
subwaydf['entries_fixed'] = subwaydf.apply(convertToMean, axis=1)

In [13]:
#create a fixed pre column and an SCP pre column for evaluating 4hr intervals below
subwaydf['fixed_pre'] = subwaydf['entries_fixed'].shift(1)
subwaydf['shift_scp'] = subwaydf['SCP'].shift(1)

In [14]:
#If the SCP is continous (same machine at a station), then evaluate if the data makes sense 
#is count increasing and total number below 4000 threshold?
#If count is decreasing, still find the difference.
#Otherwise return a null value


def calc4hr(row):
    a = row.entries_fixed
    b = row.fixed_pre
    c = row.SCP
    d = row.shift_scp
    if (a and b and (c==d)):
        if ((a>=b) & (abs(a-b) <4000)):
            return a-b
        elif ((b>a) & (abs(a-b)<4000)):
            return b-a
        else:
            return np.nan
    else:
        return None

In [15]:
#Apply the previous function to the dataframe to get 4hr entries
subwaydf['hr4'] = subwaydf.apply(calc4hr, axis=1)

In [16]:
subwaydf.columns

Index(['index', 'C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION',
       'DATE', 'TIME', 'DESC', 'ENTRIES', 'EXITS', 'datetime', 'entries_pre',
       'entries_next', 'entries_fixed', 'fixed_pre', 'shift_scp', 'hr4'],
      dtype='object')

In [17]:
cleansubway = subwaydf[['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION','datetime','hr4']]

In [18]:
cols = ['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION','datetime','hr4']

In [19]:
cols = [i.lower() for i in cols]

In [20]:
cleansubway.columns = cols

In [21]:
cleansubway.to_csv('spring_2017.csv')

In [ ]:
#subwaydf[subwaydf['STATION'] == 'BROADWAY JCT']